# I. Store Data in to descriptor

One way of using descriptor is to store data directly in its instance namespace.

## a. common caveat

Hard reference in instance namespace whichi prevent the object from being deleted by the garbage collector.

In [1]:
class IntegerValue:
    
    def __init__(self):
        self.value = {}
    
    def __get__(self, instance, owner):
        if instance is None:
            return self
        
        self.value[instance]
    
    def __set__(self, instance, value):
        self.value[instance] = int(value)

In [2]:
class Person:
    
    height = IntegerValue()
    
    def __init__(self, name):
        self.name = name
        
    def __del__(self):
        print(f"{self} is deleted ...")
        del self
    
    def __repr__(self):
        return f"{type(self).__name__}(name={self.name!r})"

In [3]:
p1 = Person('Alex')
p2 = Person('Johnny')

In [4]:
del p1

Person(name='Alex') is deleted ...


In [5]:
p1 = Person('Alex')

In [6]:
p1.height = 175.89
p2.height = 180.45

In [7]:
Person.height.value

{Person(name='Alex'): 175, Person(name='Johnny'): 180}

In [8]:
del p1

In [9]:
Person.height.value # p1 not deleted still store in instance dictionary

{Person(name='Alex'): 175, Person(name='Johnny'): 180}

### b. If object is hashable - use WeakKeyDict

In [10]:
import weakref

In [11]:
class IntegerValue:
    
    def __init__(self):
        self.value = weakref.WeakKeyDictionary()
    
    def __get__(self, instance, owner):
        if instance is None:
            return self
        
        self.value[instance]
    
    def __set__(self, instance, value):
        self.value[instance] = int(value)

In [12]:
class Person2:
    
    height = IntegerValue()
    
    def __init__(self, name):
        self.name = name
        
    def __del__(self):
        print(f"{self} is deleted ...")
        del self
    
    def __repr__(self):
        return f"{type(self).__name__}(name={self.name!r})"

In [13]:
p1 = Person2('Alex 2')
p2 = Person2('Johnny 2')

In [14]:
# d = weakref.WeakKeyDictionary()
# d[p1] = 'a'
# d.data

In [15]:
p1.height = 179.6
p2.height = 185.2

In [16]:
Person2.height.value.data

{<weakref at 0x000001C32B9AC220; to 'Person2' at 0x000001C32BB3C220>: 179,
 <weakref at 0x000001C32BB3EBD0; to 'Person2' at 0x000001C32BB3C880>: 185}

In [17]:
del p1

Person2(name='Alex 2') is deleted ...


In [18]:
Person2.height.value.data

{<weakref at 0x000001C32BB3EBD0; to 'Person2' at 0x000001C32BB3C880>: 185}

### c. If object is not hashable - cannot use WeakKeyDict

In that case we can use the id of the object. But it is of most importance to clear the instance dictionary once the object is deleted because a similar object can reuse the same id.

In [19]:
class IntegerValue:
    
    def __init__(self):
        self.value = {}
    
    def __get__(self, instance, owner):
        if instance is None:
            return self
        
        self.value[id(instance)] = int(value)
    
    def __set__(self, instance, value):
        
        # once object is deleted, its weakref is as well after the callback
        self.value[id(instance)] = (weakref.ref(instance, self._remove_object), int(value))
        
    def _remove_object(self, weak_ref): # the call back take its weak ref as argument
        print('weakref call back')        
        
        for key, val in self.value.items():
            if val[0] is weak_ref:
                del self.value[key]
                break
        pass

In [20]:
class NotHashablePerson:
    
    height = IntegerValue()
    
    def __init__(self, name):
        self.name = name
        
    def __eq__(self, other):
        return isinstance(other, type(self)) and self.name == other.namme
        
    def __del__(self):
        print(f"{self} is deleted ...")
        del self
    
    def __repr__(self):
        return f"{type(self).__name__}(name={self.name!r})"

In [21]:
not_hash_p1 = NotHashablePerson('Alex not hashable')
not_hash_p2 = NotHashablePerson('John not hashable')

In [22]:
#hash(not_hash_p1)

In [23]:
not_hash_p1.height = 175.6
not_hash_p2.height = 184.6

In [24]:
NotHashablePerson.height.value

{1937763501632: (<weakref at 0x000001C32BB41180; to 'NotHashablePerson' at 0x000001C32BB48640>,
  175),
 1937763500672: (<weakref at 0x000001C32BB49A90; to 'NotHashablePerson' at 0x000001C32BB48280>,
  184)}

In [25]:
del(not_hash_p1)

NotHashablePerson(name='Alex not hashable') is deleted ...
weakref call back


In [26]:
NotHashablePerson.height.value

{1937763500672: (<weakref at 0x000001C32BB49A90; to 'NotHashablePerson' at 0x000001C32BB48280>,
  184)}